# Quick start

Follow this guide while running all the code blocks to get a quick overview of the API capabilities. You can run code cells by selecting them and pressing the **Run** button or with the shortcut **Ctrl+ENTER**.

Before we begin:
1. Remember to replace your token in every code block where it says **MY_TOKEN**
2. **Run the following cell** once to install the Python onc client:

In [ ]:
# Install the "onc" pip package in the current Jupyter kernel (pip install onc)
import sys
!{sys.executable} -m pip install onc --upgrade

MY_TOKEN = '22ec8261-50e4-44f3-8e35-a4cf80454ff5'

## 1. Searching with discovery methods

To download ONC data, you need to specify the type of data you require and where in particular (i.e. location, device) it originates from.

In the Oceans 2.0 API, there's a unique code that identifies every location, device, property, data product type, etc. You include these codes in a group of filters that determine the data you're interested in.

Discovery methods allow you to explore the hierarchy of the ONC database to obtain the codes for your filters (they work like a "search" function).

The example below uses the getLocations method to search for locations that include _"Burrard"_ in their name (i.e. _"Burrard Inlet"_):

In [ ]:
from onc.onc import ONC
onc = ONC('22ec8261-50e4-44f3-8e35-a4cf80454ff5')

filters = {'locationName': 'Burrard'}
result = onc.getLocations(filters)
 
onc.print(result)

*Use the onc.print() method to print any result returned by the client library in a format easier to read.*

The previous code prints a list with a location that matches the search filters provided; this location includes a "locationCode" that can be used to continue searching "inside" it, as in the following examples:

In [ ]:
# what device categories are available here?
result = onc.getDeviceCategories({'locationCode': 'BIIP'})
onc.print(result)

In [ ]:
# what properties are available for the CTD category at this location?
result = onc.getProperties({'locationCode': 'BIIP', 'deviceCategoryCode': 'CTD'})
onc.print(result)

In [ ]:
# what data product types are available for the CTD category at this location?
result = onc.getDataProducts({'locationCode': 'BIIP', 'deviceCategoryCode': 'CTD'})
onc.print(result)

## 2. Obtaining sensor readings in (near) real-time

Once you determine the exact filters that identify the data you are interested in, there are 3 different methods available to download it.

One method allows you to directly download time-series of sensor readings for a given timeframe.

In the following example, we obtain 5 seconds of conductivity readings from the CTD:

In [ ]:
filters = {
  'locationCode': 'BIIP',
  'deviceCategoryCode': 'CTD',
  'propertyCode': 'conductivity',
  'dateFrom': '2019-06-20T00:00:00.000Z',
  'dateTo': '2019-06-20T00:00:05.000Z'
}
result = onc.getDirectByLocation(filters)
onc.print(result)

The result includes matching lists of "values" and "sampleTimes". We also use the property code "conductivity" to limit results to a specific property available in this CTD (where did we get this property from?).

Note that we can also get the raw, unclean readings directly from the device, using the **getDirectRawByLocation()** method:

In [ ]:
filters = {
  'locationCode': 'BIIP',
  'deviceCategoryCode': 'CTD',
  'dateFrom': '2019-06-20T00:00:00.000Z',
  'dateTo': '2019-06-20T00:00:05.000Z'
}
result = onc.getDirectRawByLocation(filters)
onc.print(result)

## 3. Downloading data products

Another method is to request the ONC servers to generate a custom data product the same way as in our **Oceans 2.0 Data Search**; This is done through the data product download methods.

The following example downloads two PNG files with plots for 30 seconds of data from a CTD (find them in the **"output"** folder of your jupyter main page):

In [ ]:
filters = {
  'locationCode': 'BIIP',
  'deviceCategoryCode': 'CTD',
  'dataProductCode': 'TSSP',
  'extension': 'png',
  'dateFrom': '2019-06-20T00:00:00.000Z',
  'dateTo': '2019-06-20T00:30:00.000Z',
  'dpo_qualityControl': '1',
  'dpo_resample': 'none'
}
result = onc.orderDataProduct(filters, includeMetadataFile=False)
 
onc.print(result)

The filters above include codes for **location**, **deviceCategory** and **dataProduct**, as well as the file **extension** and a time interval. They also include a couple of filters to configure this specific data product type (starting with the **"dpo_"** prefix) which can be obtained from the Data Product Options documentation. You can download more than 120 different types of data products including audio & video.

## 4. Downloading archived files

A faster way to download data products (if it suits your need) is to leverage how ONC scripts auto-generate and archive data products of different types at set time intervals. You can directly download these data product files from our files archive, as long as you know their unique filename.

In the following example, we get the list of archived files available for a camera at Ridley Island (in a certain time span):

In [ ]:
# get the list of files
filters = {
    'locationCode': 'RISS',
    'deviceCategoryCode': 'VIDEOCAM',
    'dateFrom': '2016-12-01T00:00:00.000Z',
    'dateTo'  : '2016-12-01T00:05:00.000Z'
}
result = onc.getListByLocation(filters, allPages=True)
onc.print(result)

Once we have the file name, you can use the method **"getFile()"** to download individual files:

In [ ]:
# download one of the files
onc.getFile('AXISQ6044PTZACCC8E334C53_201601.000Z.jpg')

You can also use the method **"getDirectFiles()"** to download all the files that match your filters.

## 5. Closing comments

There are many other methods in the client libraries that can help you grab the data you want. Take a quick look at our [Client Libraries Documentation](https://wiki.oceannetworks.ca/display/O2A/Client+Libraries) to see what's possible, or ask your Data Services Specialist if you have other questions.

Happy scripting!